{[Click aquí para leer este notebook en Google Colab](https://colab.research.google.com/drive/1ArhjRUCf_Qz3mZiwEzYR_BW5SoSSYHnL)}

<head><link rel = "stylesheet" href = "https://drive.google.com/uc?id=1zYOH-_Mb9jOjRbQmghdhsmZ2g6xAwakk"></head>

<table class = "header"><tr>
    <th align = "left">GAMBOT | Fase de desarrollo</th>
    <th align = "right">Escrito por: Gaston Solari Loudet</th>
</tr></table>

## <center><b><u>Formulación de estrategia</u></b></center>

In [ ]:
import pandas, numpy, datetime, matplotlib, matplotlib.pyplot, requests, datetime
URL = "https://drive.google.com/uc?id="
ID = "1o6V-ua4JWKU1t3bt9sFctSTjturNqwGe"
REQ = requests.get(URL + ID, allow_redirects = True)
open("./tools.py", "wb").write(REQ.content)
ID = "1TawPXELPzNlySaOx3HT67A-wHwRxuWjQ"
matplotlib.pyplot.style.use(URL + ID)
from IPython.display import display
from tools import GMAData, GMAPlot

### <center><u><b>Búsqueda de reversals</b></u></center>

#### <u><b>Generador de datos aleatorios</b></u>

In [ ]:
pmin = 1.2345  ;   nc = 5
pmnd = 0.001   ;  nb = 10
##########################
dates = datetime.datetime(2025, 10, 1)
Tix = numpy.random.randn(10*(nc + 1)*nb).cumsum()
Tix = (pmin + Tix*pmnd).round(5).reshape(-1, 10)
O, H, L = Tix[:, 0], Tix.max(1), Tix.min(1)
C = numpy.append(O[1:], Tix[-1, -1])
minute = datetime.timedelta(minutes = 1)
dates += numpy.arange(nc + 1)*minute
hour = datetime.timedelta(hours = 1)
times = numpy.arange(0, nb, 1/(nc + 1)).astype(int)
times = numpy.tile(dates, nb) + (1 + times)*hour
data = {"O": O, "H": H, "L": L, "C": C}
data = pandas.DataFrame(data, index = times)
data.iloc[:: nc + 1] = None
_, Axes = GMAPlot.candle_plot(data)
Axes.tick_params(labelsize = 12)
for b in range(nb + 1):
    Axes.axvline(x = (nc + 1)*b, color = "m")

#### <u><b>Interpoladores</b></u>

<center><img src = "https://drive.google.com/uc?id=12U2i9Q73UIRDLQzJrtXLb6JMRn38DO2_" width = "75%%"></img></center>

In [ ]:
%%capture
degree = 2
Blocks = list()
Axes.grid(False)
ymin, ymax = Axes.get_ylim()
yd = ymax - ymin
for b in range(1, len(data) + 1, nc + 1):
    t = range(b, b + nc)
    ohlc = data.iloc[t, :]
    ohlc.loc[:, "T"] = ohlc.dot([0, 1, 0, 1])/2
    poly = numpy.polyfit(t, ohlc.loc[:, "T"], degree)
    ohlc.loc[:, "P"] = numpy.polyval(poly, t)
    d1_x = numpy.polyder(poly, m = 1)
    d2_x = numpy.polyder(poly, m = 2)
    d1_v = numpy.polyval(d1_x, t)
    d2_v = numpy.polyval(d2_x, t)
    x_d0 = numpy.abs(d1_v).argmin()
    is_rev = (0 < x_d0 < nc - 1)
    curv, line = d2_v[x_d0], None
    if (is_rev and curv > 0): line = ohlc["L"].min()
    if (is_rev and curv < 0): line = ohlc["H"].max()
    Axes.plot(t, ohlc["P"], "r", lw = 3)
    Blocks.append({"ohlc": ohlc, "line": line, "curv": curv})
    if not is_rev: continue
    x, y = [t[0], t[-1]], [line, line]
    Axes.axhline(y = line, ls = ":", color = "y")
    Axes.plot(x, y, ls = "-", color = "y", lw = 2)
    Axes.text(t[0], ymin - yd/15, "K = %.4f" % curv, fontsize = 12)

In [ ]:
Axes.set_ylim(ymax = ymax, ymin = ymin - yd/10)
Axes.figure

### <center><b><u>Búsqueda de lineas de soporte y resistencia</u></b></center>

#### <b><u>Descarga de datos mediante [Finnhub.io](https://finnhub.io/docs/api/forex-candles)</u><b>

In [ ]:
import finnhub
FCL = finnhub.Client(api_key = "c1cjh0748v6vbcpf2rlg")
days = 120 ; GMT = -3  # Se banca hasta 2000 velas.
###################################################
t = datetime.datetime.now()
t = int(t.timestamp())
t0 = t - days*24*60*60

Data = FCL.forex_candles("OANDA:XAU_USD", 60, t0, t)
Data.pop("s")    ;    Data = pandas.DataFrame(Data)
Data["t"] = Data["t"] + GMT*60*60
Data.set_index("t", inplace = True)
Data.index = pandas.to_datetime(Data.index, unit = "s")
Data.columns = [_.upper() for _ in Data.columns]

Data[["H%", "L%", "C%"]] = Data.loc[:, ["H", "L", "C"]].pct_change()
Data = Data[(Data["L%"] != 0) | (Data["H%"] != 0) | (Data["C%"] != 0)] 
Figure, Axes = GMAPlot.candle_plot(Data[-480:]) ## Gráfica de velas.
Axes.set_title("Gráfica completa", fontsize = 14, fontweight = "bold");

#### <b><u>Uso de canales de máximos/mínimos móviles</u></b>

In [ ]:
period = 32
candles = 192
Data["Don.max"] = Data.loc[:, "H"].rolling(period).max()
Data["Don.min"] = Data.loc[:, "L"].rolling(period).min()
Data_reduced = Data.copy()[-candles :]
Figure, Axes = GMAPlot.candle_plot(Data_reduced)
Axes.fill_between(range(len(Data_reduced)), Data_reduced["Don.max"],
    Data_reduced["Don.min"], color = "blue", zorder = -10, alpha = 0.5);

In [ ]:
Donchian = pandas.DataFrame()
for period in range(2, len(Data_reduced)//2 + 1):
    Donchian[f"MAX_{period}"] = Data_reduced["H"].rolling(period).max()
    Donchian[f"MIN_{period}"] = Data_reduced["L"].rolling(period).min()
Donchian.index = [x.strftime(format = "%m/%d %H:%M") for x in Donchian.index]
Weights = pandas.Series(1 + numpy.arange(len(Donchian)),
          index = Donchian.index).apply(func = lambda x: x)
## Poner un dec. menos que estandar de instrumento. ej.: EURUSD = 5 - 1 = 4
Donchian = Donchian.round(4)
Donchian

#### <b><u>Lineas S&R como valores recurrentes</u></b>

In [ ]:
pc_min = 0.1 ## <== filtro de lineas
print("Dimensiones de grilla, antes: %s" % (Donchian.shape,))
Donchian = Donchian.dropna()
print("Dimensiones de grilla, despues: %s" % (Donchian.shape,))
ArrayDonchian = numpy.squeeze(Donchian.values.reshape([-1, 1]))
### De la grilla al conteo: eliminación de la linea de tiempo.
Table = pandas.value_counts(ArrayDonchian).to_frame().reset_index()
Table.rename(columns = {"index": "Lines", 0: "Count"}, inplace = True)
fmax = Table.loc[:, "Count"].max()
Table = Table[Table.iloc[:, -1] > pc_min*fmax].transpose()
Table.loc["Count", :] = Table.loc["Count", :].astype(dtype = int)
Table

In [ ]:
Figure, Axes = GMAPlot.candle_plot(Data_reduced)
fmin = Table.loc["Count", :].min()
wmin, wmax = 1, 3 ## Grosor de trazo mínimo y máximo.
## Grosor proporcional a que tan frecuente es el valor.
width = lambda x: wmin + (wmax - wmin)*(x - fmin)/(fmax - fmin)
for n in range(len(Table.columns)):
    line, freq = Table.loc[["Lines", "Count"], n]
    Axes.axhline(y = line, lw = width(freq), color = "red", alpha = 3/4)

### <center><b><u>Estrategia predictiva</u></b></center>

#### <b><u>Condiciones de ejecución</u></b>

#### <b><u>Condiciones de validación</u></b>